In [1]:
import re
from typing import Dict, List, Set

from pyswip import Prolog

In [3]:
prolog = Prolog()
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")
list(prolog.query("father(michael,X)"))  # [{'X': 'john'}, {'X': 'gina'}]

[{'X': 'john'}, {'X': 'gina'}, {'X': 'john'}, {'X': 'gina'}]

In [4]:
cryptarithms = [
    "SEND + MORE #= MONEY",
    "VIOLIN + VIOLA #= TRIO",
    "ODD + ODD =:= EVEN",
]

In [5]:
def get_vars(expr: str) -> Set[str]:
    return set(re.findall(r"[A-Z]", expr))

In [7]:
get_vars(cryptarithms[0])

{'D', 'E', 'M', 'N', 'O', 'R', 'S', 'Y'}

In [8]:
rules = [
    ":- use_module(library(clpfd))",
    "digit(X) :- member(X, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])",
    "all_diff([])",
    "all_diff([H|T]) :- \+member(H, T), all_diff(T)",
    """
    solve([[O, D, D], +, [O, D, D], =:=, [E, V, E, N]]) :-
        % génération des chiffres
        digit(O), digit(D),
        digit(E), digit(V), digit(N),
        % test que O et E sont différents de 0
        O =\= 0, E =\= 0,
        % test de la somme
        all_diff([O, D, E, V, N]),
                   100 * O + 10 * D + D +
                   100 * O + 10 * D + D =:=
        1000 * E + 100 * V + 10 * E + N
    """,
]

for rule in rules:
    prolog.assertz(rule)

In [9]:
query = "solve([[O, D, D], +, [O, D, D], =:=, [E, V, E, N]])"
list(prolog.query(query))

In [8]:
def all_digit(expr: str) -> Set[str]:
    return {f"digit({char})" for char in get_vars(expr)}

In [9]:
all_digit(cryptarithms[2])

{'digit(D)', 'digit(E)', 'digit(N)', 'digit(O)', 'digit(V)'}

In [10]:
def all_diff(expr: str) -> str:
    return f"all_diff([{', '.join(get_vars(expr))}])"

In [11]:
all_diff(cryptarithms[2])

'all_diff([N, O, E, D, V])'

In [12]:
def generate(
    expr: str, allow_zero: bool = True, allow_leading_zero: bool = False
) -> List[str]:
    generated = []
    generated += list(all_digit(expr))
    generated.append(all_diff(expr))
    if not allow_zero:
        for char in get_vars(expr):
            generated.append(f"dif({char}, 0)")
    if not allow_leading_zero:
        for char in set(re.findall(r'\b(\w)', expr)):
            generated.append(f"dif({char}, 0)")
    if allow_leading_zero:
        for char in set(re.findall(r'\b(\w)', expr)):
            if f"dif({char}, 0)" in generated:
                generated.remove(f"dif({char}, 0)")
    return generated

In [13]:
generate(cryptarithms[2], allow_zero=False, allow_leading_zero=True)

['digit(O)',
 'digit(V)',
 'digit(D)',
 'digit(E)',
 'digit(N)',
 'all_diff([N, O, E, D, V])',
 'dif(N, 0)',
 'dif(D, 0)',
 'dif(V, 0)']

In [14]:
def test(expr: str) -> str:
    operands = re.findall(r"[A-Z]+", expr)
    operators = re.findall(r"[+\-*/]|mod|#=|=:=", expr)

    rule = ""
    for i in range(len(operators)):
        rule += "("
        for char, coef in zip(operands[i], range(len(operands[i]), 0, -1)):
            rule += f"{10 ** (coef - 1)} * {char} + "
        rule = rule[:-3] + ")"
        rule += f" {operators[i]} "
    rule += "("
    for char, coef in zip(operands[-1], range(len(operands[-1]), 0, -1)):
        rule += f"{10 ** (coef - 1)} * {char} + "
    rule = rule[:-3] + ")"

    return rule

In [15]:
test(cryptarithms[2])

'(100 * O + 10 * D + 1 * D) + (100 * O + 10 * D + 1 * D) =:= (1000 * E + 100 * V + 10 * E + 1 * N)'

In [16]:
def get_operators(expr: str) -> List[str]:
    return re.findall(r"[+\-*/]|mod|#=|=:=", expr)

In [17]:
get_operators(cryptarithms[1])

['+', '#=']

In [18]:
def get_operands(expr: str) -> List[str]:
    return re.findall(r"[A-Z]+", expr)

In [19]:
get_operands(cryptarithms[2])

['ODD', 'ODD', 'EVEN']

In [20]:
def get_puzzle(expr: str) -> str:
    operators = get_operators(expr)
    operands = get_operands(expr)
    return f"solve([[{', '.join(operands[0])}], {operators[0]}, [{', '.join(operands[1])}], {operators[1]}, [{', '.join(operands[2])}]])"

In [21]:
get_puzzle(cryptarithms[1])

'solve([[V, I, O, L, I, N], +, [V, I, O, L, A], #=, [T, R, I, O]])'

In [22]:
def get_rules() -> Set[str]:
    rules = {
        ":- use_module(library(clpfd))",
        "digit(X) :- member(X, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])",
        "all_diff([])",
        "all_diff([H|T]) :- \+member(H, T), all_diff(T)",
    }
    return rules

In [23]:
def solve(
    expr: str, allow_zero: bool = True, allow_leading_zero: bool = False
) -> List[Dict[str, int]]:
    prolog = Prolog()
    for rule in get_rules():
        prolog.assertz(rule)

    query = get_puzzle(expr)
    rule = f"{query} :- "
    rule += ", ".join(map(str, generate(expr, allow_zero, allow_leading_zero)))
    rule += f", {test(expr)}"
    
    print(rule)
    print(query)
    prolog.assertz(rule)
    
    return list(prolog.query(query))

In [25]:
solve(cryptarithms[2])

solve([[O, D, D], +, [O, D, D], =:=, [E, V, E, N]]) :- digit(O), digit(V), digit(D), digit(E), digit(N), all_diff([N, O, E, D, V]), dif(O, 0), dif(E, 0), (100 * O + 10 * D + 1 * D) + (100 * O + 10 * D + 1 * D) =:= (1000 * E + 100 * V + 10 * E + 1 * N)
solve([[O, D, D], +, [O, D, D], =:=, [E, V, E, N]])
